In [10]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.
X_test = X_test / 255.

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
  model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=[X_valid_A, y_valid_A])

Epoch 1/20
1375/1375 [==============================] - 7s 4ms/step - loss: 0.6118 - accuracy: 0.7928 - val_loss: 0.3934 - val_accuracy: 0.8607
Epoch 2/20
1375/1375 [==============================] - 9s 7ms/step - loss: 0.3496 - accuracy: 0.8810 - val_loss: 0.3226 - val_accuracy: 0.8879
Epoch 3/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3119 - accuracy: 0.8929 - val_loss: 0.2955 - val_accuracy: 0.9001
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2933 - accuracy: 0.8986 - val_loss: 0.2827 - val_accuracy: 0.9036
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2805 - accuracy: 0.9034 - val_loss: 0.2721 - val_accuracy: 0.9076
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2715 - accuracy: 0.9070 - val_loss: 0.2675 - val_accuracy: 0.9076
Epoch 7/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2636 - accuracy: 0.9096 - val_loss: 0.2661 - val_accuracy:

In [ ]:
model_A.save("model_A.h5")

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
  model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=[X_valid_B, y_valid_B])

Epoch 1/20
7/7 [==============================] - 1s 45ms/step - loss: 0.9597 - accuracy: 0.3100 - val_loss: 0.7321 - val_accuracy: 0.6055
Epoch 2/20
7/7 [==============================] - 0s 31ms/step - loss: 0.6881 - accuracy: 0.6250 - val_loss: 0.5787 - val_accuracy: 0.7312
Epoch 3/20
7/7 [==============================] - 0s 18ms/step - loss: 0.5380 - accuracy: 0.7450 - val_loss: 0.4767 - val_accuracy: 0.8134
Epoch 4/20
7/7 [==============================] - 0s 15ms/step - loss: 0.4382 - accuracy: 0.8600 - val_loss: 0.4021 - val_accuracy: 0.8580
Epoch 5/20
7/7 [==============================] - 0s 17ms/step - loss: 0.3649 - accuracy: 0.9250 - val_loss: 0.3457 - val_accuracy: 0.9006
Epoch 6/20
7/7 [==============================] - 0s 17ms/step - loss: 0.3084 - accuracy: 0.9500 - val_loss: 0.3048 - val_accuracy: 0.9270
Epoch 7/20
7/7 [==============================] - 0s 16ms/step - loss: 0.2690 - accuracy: 0.9600 - val_loss: 0.2742 - val_accuracy: 0.9341
Epoch 8/20
7/7 [===========

In [ ]:
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_20 (Dense)            (None, 300)               235500    
                                                                 
 dense_21 (Dense)            (None, 100)               30100     
                                                                 
 dense_22 (Dense)            (None, 50)                5050      
                                                                 
 dense_23 (Dense)            (None, 50)                2550      
                                                                 
 dense_24 (Dense)            (None, 50)                2550      
                                                                 
 dense_25 (Dense)            (None, 1)                

In [ ]:
model_A = keras.models.load_model("model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))
# not preferred option

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 37ms/step - loss: 2.3505 - accuracy: 0.0100 - val_loss: 2.1906 - val_accuracy: 0.0507
Epoch 2/4
7/7 [==============================] - 0s 16ms/step - loss: 2.1327 - accuracy: 0.0250 - val_loss: 1.9940 - val_accuracy: 0.0730
Epoch 3/4
7/7 [==============================] - 0s 29ms/step - loss: 1.9297 - accuracy: 0.0550 - val_loss: 1.8150 - val_accuracy: 0.1034
Epoch 4/4
7/7 [==============================] - 0s 16ms/step - loss: 1.7456 - accuracy: 0.0950 - val_loss: 1.6469 - val_accuracy: 0.1359
Epoch 1/16
7/7 [==============================] - 1s 39ms/step - loss: 1.2916 - accuracy: 0.2350 - val_loss: 0.9077 - val_accuracy: 0.3671
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.7418 - accuracy: 0.5150 - val_loss: 0.6185 - val_accuracy: 0.6095
Epoch 3/16
7/7 [==============================] - 0s 17ms/step - loss: 0.5117 - accuracy: 0.7200 - val_loss: 0.4577 - val_accuracy: 0.7870
Epoch 4/16
7/7 [===============

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for i in range(20):
  model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

In [ ]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              metrics=["accuracy"], optimizer=optimizer)

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

170498071/170498071 [==============================] - 3s 0us/step


In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_model.h5", save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, "cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [14]:
%tensorboard --logdir=./cifar10_logs --port=6006

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-04-23 19:41:15.238150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.9/dist-packages/tensorboard_data_server/bin/server)
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server 

In [15]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 34s 17ms/step - loss: 3.7857 - accuracy: 0.1688 - val_loss: 2.1187 - val_accuracy: 0.2310
Epoch 2/100
1407/1407 [==============================] - 26s 19ms/step - loss: 2.0540 - accuracy: 0.2489 - val_loss: 1.9738 - val_accuracy: 0.2618
Epoch 3/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.9389 - accuracy: 0.2894 - val_loss: 1.8779 - val_accuracy: 0.3108
Epoch 4/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.8656 - accuracy: 0.3229 - val_loss: 2.0013 - val_accuracy: 0.2946
Epoch 5/100
1407/1407 [==============================] - 23s 17ms/step - loss: 1.8115 - accuracy: 0.3407 - val_loss: 1.8090 - val_accuracy: 0.3412
Epoch 6/100
1407/1407 [==============================] - 23s 17ms/step - loss: 1.7615 - accuracy: 0.3607 - val_loss: 1.7466 - val_accuracy: 0.3630
Epoch 7/100
1407/1407 [==============================] - 24s 17ms/step - loss: 1.7172 - accuracy: 0.3792 - val_loss: 1

In [16]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 1s 4ms/step - loss: 1.5850 - accuracy: 0.4692


[1.5850058794021606, 0.4691999852657318]

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 58s 26ms/step - loss: 1.8388 - accuracy: 0.3410 - val_loss: 1.6494 - val_accuracy: 0.4066
Epoch 2/100
1407/1407 [==============================] - 37s 26ms/step - loss: 1.6652 - accuracy: 0.4058 - val_loss: 1.5549 - val_accuracy: 0.4408
Epoch 3/100
1407/1407 [==============================] - 37s 26ms/step - loss: 1.5999 - accuracy: 0.4305 - val_loss: 1.5079 - val_accuracy: 0.4544
Epoch 4/100
1407/1407 [==============================] - 36s 26ms/step - loss: 1.5503 - accuracy: 0.4488 - val_loss: 1.5173 - val_accuracy: 0.4560
Epoch 5/100
1407/1407 [==============================] - 37s 26ms/step - loss: 1.5065 - accuracy: 0.4663 - val_loss: 1.4460 - val_accuracy: 0.4870
Epoch 6/100
1407/1407 [==============================] - 36s 25ms/step - loss: 1.4697 - accuracy: 0.4803 - val_loss: 1.4295 - val_accuracy: 0.4948
Epoch 7/100
1407/1407 [==============================] - 37s 26ms/step - loss: 1.4347 - accuracy: 0.4907 - val_loss: 1

[1.3141878843307495, 0.5329999923706055]

In [18]:
model.evaluate(X_valid_scaled, y_valid)


NameError: ignored